# Headers/Startup

In [ ]:
import requests
from urllib.parse import urlencode
import json
import logging
import traceback
from os import environ as os_environ
from sys import stdout
from sqlalchemy.dialects.postgresql import BIGINT, INTEGER, ARRAY, TIMESTAMP, VARCHAR


import dotenv

dotenv.load_dotenv()

from datetime import date, datetime, timedelta

import pandas as pd

# for verifying DB updates
from db_engines import mms_db, rprt_db, wh_db as db

tmstmp_fmt = r'%Y-%m-%d %H:%M:%S'
query_date_fmt = r'%Y-%m-%d'
file_name_date_fmt = r'%Y%m%d'

import re
from pathlib import Path

import pandas as pd
from pandas import DataFrame as Df
from pandas import Series as Ser

today: str = datetime.now().strftime(query_date_fmt)


In [ ]:
log_fmt_date_strm = r'%y%m%d|%H%M'
log_fmt_date_file = r'%Y-%m-%d %H:%M:%S'
log_fmt_file = '%(asctime)s [%(name)s,%(funcName)s,%(module)s::%(levelname)s]>>%(message)s'
log_fmt_strm = '\x1b[32m%(asctime)s[%(name)s %(levelname)s]\x1b[0m >> %(message)s'

logger = logging.getLogger(os_environ['PRMDIA_MM_LOGNAME'])
hdlr = logging.StreamHandler(stdout)
hdlr.setFormatter(logging.Formatter(fmt=log_fmt_strm, datefmt=log_fmt_date_strm))
# hdlr.setLevel(logging.DEBUG)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)

In [ ]:
# yr_min, month_min, day_min = 2022, 10, 28  # yesterday or fri before mon
# yr_lmt, month_lmt, day_lmt = 2022, 11, 1  # usually today

lmt_date = datetime.now().date()

# days before limit date (usually today)
min_date: datetime = datetime(2022, 11, 28)

day_range = 3

# # comment out line below to use the explicit date
# min_date: datetime = lmt_date - timedelta(days=day_range)
max_date: datetime = lmt_date - timedelta(days=1)


date_min: str
date_lmt: str
date_max: str
date_min, date_lmt, date_max = (
        d.strftime(query_date_fmt)
        for d in (min_date, lmt_date, max_date)
    )
date_min_strrep, date_lmt_strrep, date_max_strrep = (
        s.replace('-', '.')
        for s in
        (date_min, date_lmt, date_max)
    )
logger.debug(f">={date_min_strrep}, < {date_lmt_strrep}, <= {date_max_strrep}")

In [ ]:
# output config
client_lead_cols = json.loads(Path('client_lead_cols.json').read_text())

type_map = {
    'BIGINT': BIGINT,
    'INT': INTEGER,
    'VARCHAR[]': ARRAY(VARCHAR),
    'TIMESTAMP': TIMESTAMP,
}

dtype = {
    k: type_map[d['dtype']]
    for k, d in client_lead_cols.items()
    if d['dtype']
}

out_fn = f'RPRT.MEDMSTR-backup.{min_date.strftime(file_name_date_fmt)}_{max_date.strftime(file_name_date_fmt)}'
out_pth = Path('sh') / 'local_repos' / 'RPRT.MEDMSTR-BACKUP'


In [ ]:
# VICI CONFIGS
base_url = 'http://10.1.10.20/vicidial/non_agent_api.php'
static_params: dict[str, str|int|bool] = {
    'user': 6666,
    'pass': 'RedLakeSky3501',
    'function': 'add_lead',
    'custom_fields': 'Y',
    'duplicate_check': 'DUPLIST30DAY',
    'source': __name__
}
var_params: dict[str, str|int|bool] = {
    'list_id': None,
    'phone_number': None,
    'STA': None
}
cmpgn_lists: dict = {
    int(k): int(v) for k, v in
    json.loads(
        Path('campaign_lists.json')
        .read_text()
    ).items()
}


In [ ]:
# check for active connections, else raise exception and bail
from db_engines import MySQL_OpErr, check_connection

for d in db, rprt_db:
    try:
        check_connection(d)
    except MySQL_OpErr:
        raise Exception(f"\x1b[91mSEE BELOW/ABOVE\x1b[0m\n")
    else:
        pass

del MySQL_OpErr, check_connection


# Queries

In [ ]:
df: Df
with db.connect() as conn:
    df = (
        pd.read_sql_query(
            f"""--sql
                SELECT * FROM med_master_join
                WHERE att_connected < '{date_lmt}'
                AND att_connected >= '{date_min}'
                ;
            """.replace('--sql\n', ''),
            conn
        )
        .convert_dtypes()
    )

In [ ]:
# assign delivery dates and AF code to AF deliverables
split_truth: Ser = df['af_client'].notna()


df['DELIVERY_DATE'] = df['af_connected']

df.loc[split_truth, ['DELIVERY_CODE']] = 'AF'

df.loc[~split_truth, ['DELIVERY_DATE']] = pd.NA
df.loc[~split_truth, ['DELIVERY_CODE']] = pd.NA

df['PM_COMMENTS'] = pd.NA



In [ ]:
# load af billables to table
load = (
    df.loc[
        # has delivery code
        df['DELIVERY_CODE'].notna(),
        # select only needed columns
        [d['orig'] for d in client_lead_cols.values()]
    ]
    .rename(columns={
        d['orig']: k for k, d in
        client_lead_cols.items()
        if d['orig']
    })
    .astype({
        k: d['astype'] for k, d in
        client_lead_cols.items()
        if d['astype']
    })
)
with db.connect() as conn:
    load.to_sql(
        name='billables',
        con=conn,
        index=False,
        if_exists='append',
        dtype=dtype
    )

    # delete any duplicates appended
    conn.execute(
        statement="""--sql
            DELETE FROM
                billables a
                    USING billables b
            WHERE
                a.id < b.id
                AND a.toll_dialed = b.toll_dialed
                AND a.client_callerid = b.client_callerid
                AND a.lead_delivery_date = b.lead_delivery_date
            ;
        """.replace('--sql\n', '')
    )


In [ ]:
# load to vici
# ***ED 105, Neuro 106, and Knee/WtLoss 107 ONLY UNTIL THE OTHER LISTS ARE READY***

# where there is no delivery code
for r in df.loc[~df['DELIVERY_CODE'].notna()].itertuples():
    ph: int = r.att_callerid
    # campaign id
    cp: int = r.att_acct_af
    list_id: int = cmpgn_lists[cp]

    # caller state
    st: str = r.att_state_agg[0]  # re.sub(r'[\[\]]', '', r.att_state_agg)

    args = {
        'list_id': list_id,
        'phone_number': ph,
        'STA': st
    }
    
    # combine static args with variable args
    if list_id != 108:
        u = f"{base_url}?{urlencode(static_params | args)}"

        logger.debug(f"{u}")

        resp = requests.get(u)
        logger.info(f"ATT Orig. Ph.: {ph} Campaign ID: {cp} LIST ID: {list_id} ATT state: {st} >> {resp}")
        resp_log = f"resp: {resp} >> {resp.text}"
        if re.findall('ERROR', resp.text):
            logger.error(f"\t" + resp_log.replace('\n', '-|'))
        else: logger.debug(resp_log)

        # TODO: #3 #2 ADD COUNTER FOR GOOD RESPONSE, BAD RESPONSE, 200, 400, ETC
